# Investigate

In [1]:
import os
import onnx

model_name = "qwen25_vli_visual.cpu.float32.onnx-dynamo.onnx"
print(os.path.abspath(model_name))
model_onnx = onnx.load(model_name)

model_name_custom = "qwen25_vli_visual.cpu.float32.custom.onnx"
print(os.path.abspath(model_name_custom))
model_onnx_custom = onnx.load(model_name_custom)

/home/xadupre/github/onnx-diagnostic/_scripts/qwen25_vli_visual.cpu.float32.onnx-dynamo.onnx
/home/xadupre/github/onnx-diagnostic/_scripts/qwen25_vli_visual.cpu.float32.custom.onnx


In [2]:
from onnx_diagnostic.helpers.onnx_helper import pretty_onnx

for node in model_onnx.graph.node:
    if node.name == "n16":
        print(pretty_onnx(node))
        g = node.attribute[0].g
        break

for node in model_onnx_custom.graph.node:
    if node.name == "n16":
        print(pretty_onnx(node))
        g_custom = node.attribute[0].g
        break

Loop(num_patches, , seq_attn) -> seq_attn_6
Loop(num_patches, , seq_attn) -> seq_attn_6


In [3]:
g.input

[name: "i_patch"
type {
  tensor_type {
    elem_type: 7
    shape {
    }
  }
}
, name: "cond_in"
type {
  tensor_type {
    elem_type: 9
    shape {
    }
  }
}
, name: "seq_attn_3"
type {
  tensor_type {
    elem_type: 1
  }
}
]

In [4]:
g.output

[name: "cond_out"
type {
  tensor_type {
    elem_type: 9
    shape {
    }
  }
}
, name: "seq_attn_5"
]

In [5]:
g_custom.input

[name: "i_patch"
type {
  tensor_type {
    elem_type: 7
    shape {
    }
  }
}
, name: "cond_in"
type {
  tensor_type {
    elem_type: 9
    shape {
    }
  }
}
, name: "seq_attn_3"
type {
}
]

In [6]:
g_custom.output

[name: "cond_out"
type {
  tensor_type {
    elem_type: 9
    shape {
    }
  }
}
, name: "seq_attn_5"
type {
}
]

In [ ]:
def remove_inplace_body_last_input_output_type_for_loop(filename: str):
    model = onnx.load(filename, load_external_data=False)
    for node in model.graph.node:
        if node.op_type == "Loop":
            g = node.attribute[0].g
            g.input[-1].type.CopyFrom(onnx.TypeProto())
            g.output[-1].type.CopyFrom(onnx.TypeProto())


remove_inplace_body_last_input_output_type_for_loop(model_name)

tensor_type {
  elem_type: 1
}
 ['ByteSize', 'Clear', 'ClearExtension', 'ClearField', 'CopyFrom', 'DESCRIPTOR', 'DiscardUnknownFields', 'FindInitializationErrors', 'FromString', 'HasExtension', 'HasField', 'IsInitialized', 'ListFields', 'Map', 'MergeFrom', 'MergeFromString', 'Opaque', 'Optional', 'ParseFromString', 'Sequence', 'SerializePartialToString', 'SerializeToString', 'SetInParent', 'SparseTensor', 'Tensor', 'UnknownFields', 'WhichOneof', '_CheckCalledFromGeneratedFile', '_ListFieldsItemKey', '_SetListener', '__class__', '__contains__', '__deepcopy__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__unicode__', 'denotation', 'map_type', 'opaque_type', 'optional_type', 'sequence_ty

AttributeError: copy_from